# 데이터 수집 및 전처리

## 데이터 수집
**사용 데이터셋**
- [감성 대화 말뭉치](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86)
- [감정 분류를 위한 대화 음성 데이터셋](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=263)

'감정 대화 말뭉치' 데이터셋을 기반으로 학습을 진행 계획했지만, 중립적(Neutral)인 감정 데이터가 존재하지 않지 때문에 '감정 분류를 위한 대화 음성 데이터셋'에서 중립적인 데이터를 수집하여 통합하였다.

In [18]:
import pandas as pd

file_neutral1 = './src/data/4차년도.csv'
file_neutral2 = './src/data/5차년도.csv'
file_neutral3 = './src/data/5차년도_2차.csv'
file_train = './src/data/감성대화말뭉치(최종데이터)_Training.xlsx'
file_validation = './src/data/감성대화말뭉치(최종데이터)_Validation.xlsx'

### 중립 데이터
감정 분류를 위한 대화 음성 데이터셋

- 4차년도 14,606건
- 5차년도 10,011건
- 5차년도(2차) 19,374건

In [19]:
# 데이터 통합 및 확인
data_neutral1 = pd.read_csv(file_neutral1, encoding='cp949')
data_neutral2 = pd.read_csv(file_neutral2, encoding='cp949')
data_neutral3 = pd.read_csv(file_neutral3, encoding='cp949')

data_neutral = pd.concat([data_neutral1, data_neutral2, data_neutral3])
data_neutral

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5e258fd1305bcf3ad153a6a4,"어, 청소 니가 대신 해 줘!",anger,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male
1,5e258fe2305bcf3ad153a6a5,둘 다 청소 하기 싫어. 귀찮아.,anger,Neutral,0,Angry,1,Neutral,0,Neutral,0,Angry,1,27,male
2,5e258ff5305bcf3ad153a6a6,둘 다 하기 싫어서 화내.,anger,Angry,1,Angry,1,Neutral,0,Angry,1,Angry,1,27,male
3,5e25902f305bcf3ad153a6a9,그럼 방세는 어떡해.,anger,Sadness,1,Sadness,1,Sadness,1,Sadness,1,Sadness,1,27,male
4,5e27f90b5807b852d9e0157b,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,sad,Sadness,1,Sadness,1,Sadness,1,Sadness,2,Sadness,1,32,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19369,5fbe313c44697678c497c05a,나 엘리베이터에 갇혔어.,fear,happiness,1,sadness,1,sadness,2,sadness,1,sadness,1,23,female
19370,5fbe251044697678c497bfb8,하지만 기분이 나쁜 걸 어떡해?,angry,sadness,1,fear,1,sadness,2,sadness,1,neutral,0,23,female
19371,5fbe31584c55eb78bd7cee7f,자취방 엘리베이턴데 정전인가봐.,fear,sadness,1,neutral,0,sadness,2,fear,1,sadness,1,23,female
19372,5fbe2f8544697678c497c047,나 드디어 프로젝트 끝났어!,happiness,disgust,1,sadness,1,neutral,0,happiness,1,sadness,1,23,female


In [20]:
# Neutral 데이터만 추출
# 데이터 년도에 따라 Neutral, neutral 둘다 존재 주의
data_neutral_text = data_neutral[(data_neutral['1번 감정'] == 'neutral') | (data_neutral['1번 감정'] == 'Neutral')]['발화문']  # extract only neutral text
data_neutral_text.duplicated().sum()  # 중복 데이터 확인

1054

In [21]:
# 중복 제거
data_neutral_text = data_neutral_text.drop_duplicates()
data_neutral_text

0                     어, 청소 니가 대신 해 줘!
1                   둘 다 청소 하기 싫어. 귀찮아.
5                            그냥 걷고 있어.
7        처음 학원에서 만났다가 서로 좋아해서 사귀게 되었지.
8           내가 애정 표현을 잘 못해서 자주 싸우긴 했어.
                     ...              
19352          방향제도 뿌려봤는데 냄새만 더 이상해졌어.
19354                     담엔 꼭 그렇게 할게.
19359             무슨 일이 있는지 경찰들이 많더라고.
19364      밖에도 못나가고 집에서 일만 하니까 너무 힘들어.
19367                   어떤 홈 트레이닝이 있어?
Name: 발화문, Length: 10904, dtype: object

## 감정 데이터
분노, 슬픔, 불안, 상처, 당황, 기쁨 6가지 대분류로 분류 

In [22]:
# Train, Validation을 통합하여 이후 다시 재구축
data_train = pd.read_excel(file_train)
data_validation = pd.read_excel(file_validation)
data_emotion = pd.concat([data_train, data_validation])
data_emotion

,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6636,6637,청년,여성,"연애,결혼,출산",해당없음,당황,열등감,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...,같은 시기에 결혼하는 친구들끼리 상황이 다르다 보니 마음이 좋지 않으시겠어요.,우리만 뒤처지는 것 같고. 그래도 열심히 살다 보면 우리도 집을 살 수 있겠지?,우리 부부만 뒤처지는 것 같아 속상하지만 열심히 살다 보면 집을 살 수 있다고 믿으...,NaN,NaN
6637,6638,청년,여성,"연애,결혼,출산",해당없음,당황,열등감,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아.,기분이 좋지 않으신 이유를 알고 계시나요?,나도 결혼했지만 아이도 생기지 않고 그 정도로 행복하지 않거든. 친구보다 못사는 것...,친구분과 비교하게 되어 기분이 좋지 않으셨군요. 어떻게 하면 기분이 풀릴 수 있을까요?,NaN,NaN
6638,6639,청년,여성,"연애,결혼,출산",해당없음,당황,열등감,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.,돈을 모으는 것으로 자괴감이 드시는군요.,요새 집값이 너무 올라서 한숨만 나와.,집값이 올라 한숨만 나오시는군요.,NaN,NaN
6639,6640,청년,여성,"연애,결혼,출산",해당없음,당황,열등감,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...,먼저 결혼한 친구가 부럽군요.,맞아. 그 친구와 비교하게 되니 자존감이 낮아지는 기분이야.,비교로 자존감이 낮아지는군요.,NaN,NaN


In [23]:
# 데이터 분포 확인
data_emotion['감정_대분류'].value_counts()

감정_대분류
불안    10433
분노    10417
상처    10150
슬픔    10128
당황     9804
기쁨     7339
Name: count, dtype: int64

In [24]:
data_emotion['감정_소분류'].value_counts()

감정_소분류
고립된            2007
혼란스러운          1988
걱정스러운          1336
짜증내는           1179
분노             1141
두려운            1126
스트레스 받는        1119
눈물이 나는         1114
우울한            1109
노여워하는          1099
억울한            1082
툴툴대는           1053
성가신            1051
괴로워하는          1049
한심한            1044
슬픔             1044
충격 받은          1041
상처             1028
불안             1025
실망한            1022
가난한, 불우한       1016
후회되는           1013
구역질 나는         1012
죄책감의           1010
부끄러운           1010
좌절한            1008
초조한            1004
질투하는           1002
비통한            1002
안달하는           1002
환멸을 느끼는         999
배신당한            994
당황              993
낙담한             992
외로운             992
악의적인            978
당혹스러운           976
남의 시선을 의식하는     976
조심스러운           972
버려진             953
회의적인            948
희생된             942
취약한             941
마비된             921
염세적인            912
열등감             907
혐오스러운           906
방어적인            894
만족스러운           779
편안한          

In [25]:
# 필요 데이터만 추출
data_emotion_text = data_emotion[['감정_대분류', '사람문장1']]
data_emotion_text

,감정_대분류,사람문장1
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
...,...,...
6636,당황,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...
6637,당황,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아.
6638,당황,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.
6639,당황,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...


#### 데이터 비율 재조정
기쁨 감정은 데이터 전부 이용하며 부정적 감정은 50%만 이용

In [26]:
data_negative = data_emotion_text[data_emotion_text['감정_대분류'] != '기쁨']
data_positive = data_emotion_text[data_emotion_text['감정_대분류'] == '기쁨']
data_negative.info()
data_positive.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50932 entries, 0 to 6640
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   감정_대분류  50932 non-null  object
 1   사람문장1   50932 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 7339 entries, 7 to 6631
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   감정_대분류  7339 non-null   object
 1   사람문장1   7339 non-null   object
dtypes: object(2)
memory usage: 172.0+ KB


In [27]:
grouped = data_negative.groupby('감정_대분류', group_keys=False)
data_negative_reduced = grouped.apply(lambda x: x.sample(frac=0.5)).reset_index(drop=True)
data_negative_reduced

C:\Users\Chris\AppData\Local\Temp\ipykernel_1672\2119534794.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_negative_reduced = grouped.apply(lambda x: x.sample(frac=0.5)).reset_index(drop=True)


,감정_대분류,사람문장1
0,당황,어머니도 아버지도 키가 백칠십이 넘는데 나만 키가 작아. 난 입양아가 아닐까?
1,당황,나 이번에 내가 원하는 회사에 취업했어!
2,당황,누나는 항상 자기가 쓴 물건이나 먹은 음식을 스스로 치울 생각을 안 해서 한심하게 ...
3,당황,내 친구는 항상 명품만 입는데 카드빚도 많아. 그런 걸 보면 좀 한심해.
4,당황,아빠 지갑에서 돈을 조금 훔쳤는데 아빠가 나를 대하는 행동이 달라져서 당황스러워.
...,...,...
25460,슬픔,난 진지하게 고민을 말했는데 친구가 장난이라고 생각한 것 같아서 실망스러워.
25461,슬픔,건강에 신경쓰는 것도 이제 피곤해 너무 지치고 힘들어. 삶을 더 이렇게 아픈 거에 ...
25462,슬픔,이번 승진에서 탈락했어. 나도 그동안 진짜 열심히 일했는데.
25463,슬픔,그동안 신경을 쓰지 못해 건강이 많이 나빠졌어.


In [28]:
# 절반으로 감소한 것 확인
data_negative_reduced['감정_대분류'].value_counts()

감정_대분류
불안    5216
분노    5208
상처    5075
슬픔    5064
당황    4902
Name: count, dtype: int64

In [29]:
data_emotion_reduced_text = pd.concat([data_negative_reduced, data_positive])
data_emotion_reduced_text

,감정_대분류,사람문장1
0,당황,어머니도 아버지도 키가 백칠십이 넘는데 나만 키가 작아. 난 입양아가 아닐까?
1,당황,나 이번에 내가 원하는 회사에 취업했어!
2,당황,누나는 항상 자기가 쓴 물건이나 먹은 음식을 스스로 치울 생각을 안 해서 한심하게 ...
3,당황,내 친구는 항상 명품만 입는데 카드빚도 많아. 그런 걸 보면 좀 한심해.
4,당황,아빠 지갑에서 돈을 조금 훔쳤는데 아빠가 나를 대하는 행동이 달라져서 당황스러워.
...,...,...
6625,기쁨,입사 초반에는 인간관계가 힘들었는데 이제 좀 편안해진 것 같아.
6628,기쁨,빨리 결혼식을 하고 싶어.
6629,기쁨,이번에 무사히 출산했어. 정말 다행이야.
6630,기쁨,아이가 뛰어놀다가 넘어져서 깜짝 놀랐었어.


In [47]:
# 감정 데이터와 중립 데이터 통합

def merge_emotion_data(data_emotion, data_neutral):
    '''
    data_emotion: ['감정_대분류', '사람문장1']
    data_neutral: vector(text only)
    '''
    
    data_renamed = data_emotion.rename(columns={'감정_대분류': 'emotion', '사람문장1': 'text'})
    
    data_neutral_df = data_neutral.to_frame(name='text')
    data_neutral_df['emotion'] = '중립'
    
    result = pd.concat([data_renamed, data_neutral_df], ignore_index=True)

    return result

In [48]:
data_reduced = merge_emotion_data(data_emotion_reduced_text, data_neutral_text)
data_all = merge_emotion_data(data_emotion_text, data_neutral_text)

In [49]:
data_reduced['emotion'].value_counts()

emotion
중립    10905
기쁨     7339
불안     5216
분노     5208
상처     5075
슬픔     5064
당황     4902
Name: count, dtype: int64

In [50]:
# 데이터 저장
from sklearn.model_selection import train_test_split
import os

def save_data(data, path, train=0.8, train_file='train.csv', test_file='test.csv'):
    # Split the data
    train_data, test_data = train_test_split(data, train_size=train, random_state=42, shuffle=True)
    
    # Create the directory if it does not exist
    os.makedirs(path, exist_ok=True)
    
    # Save the data to CSV files
    train_data.to_csv(os.path.join(path, train_file), index=False)
    test_data.to_csv(os.path.join(path, test_file), index=False)

In [51]:
save_data(data_reduced, './src/data')
save_data(data_all, './src/data', train_file='train_all.csv', test_file='test_all.csv')